In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [34]:
class NiniSiteFroumCrawler():
    
    def __init__(self,forum_number):
        self.main_link = "https://www.ninisite.com"
        self.forum_link = self.main_link+'/discussion/forum/'+str(forum_number)
        self.initial_response = requests.get(self.forum_link)
        
        
    def get_number_of_pages_in_forum(self):
        response = self.initial_response
        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html,'html.parser')
            self.number_of_pages = int(soup.find_all('div',{'class':'text-xs-center text-sm-left'})[0].find_all('li')[-2].text)
        return self.number_of_pages
    
    
    def topic_header_parser(self, topic):
        parsed_topic = {
            'subject' : topic.find('span',{'class':'topic_subject'}).text,
            'link' : self.main_link+topic.find('a')['href'],
            'creator_username' : topic.find('div',{'class':'col-xs-12 p-x-0 pull-xs-right last-topic-user hidden-sm-down'}).text.split()[-1],
            'creator_userid' : topic.find('div',{'class':'col-xs-12 p-x-0 pull-xs-right last-topic-user hidden-sm-down'}).find('a')['href'].split('/')[2],
            'number_of_replies' : int(topic.find('span',{'class':'topic_number hidden-sm-up'}).text.split()[0])
        }
        return parsed_topic
    
    
    def crawl_topics_in_forum(self,starting_page=1,number_pages=10):
        self.topics_data = []
        for pg_number in range(starting_page,number_pages):
            print(pg_number)
            html = requests.get(self.forum_link+f'?page={pg_number}').text
            soup = BeautifulSoup(html,'html.parser')
            topics = soup.find_all('div',{'class':'col-xs-12 category--header p-x-0'})
            for topic in topics:
                self.topics_data.append(self.topic_header_parser(topic))
                
            time.sleep(1.5) 
        return pd.DataFrame(self.topics_data)

In [35]:
crawler = NiniSiteFroumCrawler(141)
topics = crawler.crawl_topics_in_forum(starting_page=5,number_pages=6)

5


In [36]:
topics

,subject,link,creator_username,creator_userid,number_of_replies
0,گروه خونی B+ ها اعلام حضور کنن,https://www.ninisite.com/discussion/topic/1117...,کنکورکنکور,1e83199d-ce1a-4dd9-8f28-ffba94cd3522,33
1,اگر دامادی برادرتون باشه چی میپوشین؟,https://www.ninisite.com/discussion/topic/1117...,nasrinkhatoon,8eb5b1f4-b420-4149-a7e0-449b279c5cea,52
2,ارتباطبا خانواده شوهر,https://www.ninisite.com/discussion/topic/1117...,مارحامی,093604c7-65da-4b83-9575-87617e86e84f,8
3,محبت می‌کنین سوره کوثر یا حمد بخونین واسه بچه ...,https://www.ninisite.com/discussion/topic/1117...,بهار۳۰۳۸,08040371-f94d-4479-86d7-a802f649506c,5
4,کسی از طب سنتی درمان شده ؟؟؟,https://www.ninisite.com/discussion/topic/1117...,rahaabedi00,db787220-e290-48ba-b482-7edc4a2c3b17,1
5,کسی بوده پسره اول پسش بزنه بعد عاشقش بشع؟؟؟؟,https://www.ninisite.com/discussion/topic/1117...,گلسرخما,f5334199-7122-4667-8452-03710dabe4df,9
6,کمکم کنید,https://www.ninisite.com/discussion/topic/1117...,فقطخداااااااا,746f7f09-827d-4a75-945f-1d54684a9545,0
7,اولین قرار عاشقانه ما....«تاپیکی که همه بایدن ...,https://www.ninisite.com/discussion/topic/1117...,mm1379a,25bb1b72-5d88-449a-a19f-246acbe7ef4d,86
8,امام رضا,https://www.ninisite.com/discussion/topic/1061...,501,7ea3a3fe-0d44-4868-9b88-5ac1e6fbea07,52
9,دوستان کرمانی راهنمایی میخوام,https://www.ninisite.com/discussion/topic/1117...,mamanarmea,ac8c5a79-d113-4175-bbc1-ab5b56c753ec,5
